# Evaluación de Modelos LLM en Modismos Colombianos

Este notebook evalúa qué tan bien los modelos LLM entienden los modismos colombianos usando:
- **BERTScore**: Para comparar similitud semántica entre textos (definiciones)
- **Accuracy/Exact Match**: Para clasificación y coincidencias exactas

In [6]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Estilos para gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Modelos
MODEL_NAMES = ["amazon/nova-micro-v1", "cohere/command-r-08-2024"]

# Directorios
DATA_DIR = 'Metricas LLM/Data_for_Metrics'
OUTPUT_DIR = 'Metricas LLM/Metricas_Resultados'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Datos en: {DATA_DIR}")
print(f"Resultados en: {OUTPUT_DIR}")
print(f"Modelos: {MODEL_NAMES}")

Datos en: Metricas LLM/Data_for_Metrics
Resultados en: Metricas LLM/Metricas_Resultados
Modelos: ['amazon/nova-micro-v1', 'cohere/command-r-08-2024']


In [7]:
# Importar funciones de métricas
import sys
sys.path.append('Metricas LLM')

from BertScore import compute_bertscore_beto, compute_bertscore_sci_beto
from SentenceBert import compute_sbert_similarity
from chrF import compute_chrf_batch

print("✓ Métricas cargadas correctamente:")
print("  - BERTScore (BETO y SciBETO)")
print("  - Sentence-BERT (paraphrase-multilingual-mpnet-base-v2)")
print("  - chrF (Character n-gram F-score)")

✓ Métricas cargadas correctamente:
  - BERTScore (BETO y SciBETO)
  - Sentence-BERT (paraphrase-multilingual-mpnet-base-v2)
  - chrF (Character n-gram F-score)


## 1. PROMPT 1: Modismo → Definición

**Métrica**: BERTScore (F1)  
**Justificación**: Evalúa similitud semántica entre la definición generada y la real. Ideal para entender si el modelo captura el significado del modismo.

In [8]:
print("="*80)
print("PROMPT 1: Modismo → Definición")
print("="*80)
print("Objetivo: Evaluar si el modelo puede generar una definición correcta del modismo")
print("Métricas:")
print("  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT")
print("  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet")
print("  - chrF: Character n-gram F-score")
print("-"*80)

# Cargar datos desde JSON
with open(os.path.join(DATA_DIR, 'prompt_1_metrics_data.json'), 'r', encoding='utf-8') as f:
    data_p1 = json.load(f)

# Filtrar datos vacíos
data_p1_valid = [d for d in data_p1 if d.get('definicion_real') and d.get('definicion_generada')]

print(f"Datos cargados: {len(data_p1_valid)} registros válidos de {len(data_p1)} totales")
print()

# ============================================================================
# 1. BERTScore con BETO y SciBETO
# ============================================================================
bert_models = [
    ('BETO', compute_bertscore_beto),
    ('SciBETO', compute_bertscore_sci_beto)
]

for bert_name, bert_func in bert_models:
    print(f"\n{'='*60}")
    print(f"Calculando BERTScore con {bert_name}")
    print('='*60)
    
    resultados_p1 = []
    
    for model in MODEL_NAMES:
        print(f"Evaluando modelo: {model}")
        
        # Filtrar datos de este modelo
        model_data = [d for d in data_p1_valid if d.get('modelo') == model]
        
        if not model_data:
            print(f"  ⚠ No hay datos para {model}")
            continue
        
        referencias = [d['definicion_real'] for d in model_data]
        candidatos = [d['definicion_generada'] for d in model_data]
        
        # BERTScore: compara candidatos (generados) vs referencias (reales)
        P, R, F1 = bert_func(candidatos, referencias)
        
        for idx, (p, r, f1) in enumerate(zip(P.tolist(), R.tolist(), F1.tolist())):
            resultados_p1.append({
                'modismo': model_data[idx]['modismo'],
                'modelo': model,
                'bert_model': bert_name,
                'precision': p,
                'recall': r,
                'f1_score': f1,
                'definicion_real': referencias[idx],
                'definicion_generada': candidatos[idx]
            })
    
    # Guardar resultados
    output_file = os.path.join(OUTPUT_DIR, f'prompt_1_{bert_name.lower()}_bertscore_resultados.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(resultados_p1, f, ensure_ascii=False, indent=2)
    
    print()
    print(f"RESULTADOS con {bert_name}:")
    for model in MODEL_NAMES:
        model_results = [r for r in resultados_p1 if r['modelo'] == model]
        if model_results:
            f1_scores = [r['f1_score'] for r in model_results]
            precisions = [r['precision'] for r in model_results]
            recalls = [r['recall'] for r in model_results]
            
            f1_mean = np.mean(f1_scores)
            f1_std = np.std(f1_scores)
            p_mean = np.mean(precisions)
            r_mean = np.mean(recalls)
            
            print(f"   {model}:")
            print(f"      • Precision: {p_mean:.4f}")
            print(f"      • Recall:    {r_mean:.4f}")
            print(f"      • F1 Score:  {f1_mean:.4f} (±{f1_std:.4f})")
    
    print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 2. Sentence-BERT Similarity
# ============================================================================
print(f"\n{'='*60}")
print(f"Calculando Sentence-BERT Similarity")
print('='*60)

resultados_sbert = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")
    
    model_data = [d for d in data_p1_valid if d.get('modelo') == model]
    
    if not model_data:
        print(f"  ⚠ No hay datos para {model}")
        continue
    
    referencias = [d['definicion_real'] for d in model_data]
    candidatos = [d['definicion_generada'] for d in model_data]
    
    # Calcular similitud con Sentence-BERT
    similarities = compute_sbert_similarity(candidatos, referencias)
    
    for idx, sim in enumerate(similarities):
        resultados_sbert.append({
            'modismo': model_data[idx]['modismo'],
            'modelo': model,
            'similarity': float(sim),
            'definicion_real': referencias[idx],
            'definicion_generada': candidatos[idx]
        })

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_1_sbert_similarity_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_sbert, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS con Sentence-BERT:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_sbert if r['modelo'] == model]
    if model_results:
        similarities = [r['similarity'] for r in model_results]
        sim_mean = np.mean(similarities)
        sim_std = np.std(similarities)
        
        print(f"   {model}:")
        print(f"      • Similitud: {sim_mean:.4f} (±{sim_std:.4f})")

print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 3. chrF Score
# ============================================================================
print(f"\n{'='*60}")
print(f"Calculando chrF Score")
print('='*60)

resultados_chrf = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")
    
    model_data = [d for d in data_p1_valid if d.get('modelo') == model]
    
    if not model_data:
        print(f"  ⚠ No hay datos para {model}")
        continue
    
    referencias = [d['definicion_real'] for d in model_data]
    candidatos = [d['definicion_generada'] for d in model_data]
    
    # Calcular chrF
    chrf_scores = compute_chrf_batch(candidatos, referencias)
    
    for idx, score in enumerate(chrf_scores):
        resultados_chrf.append({
            'modismo': model_data[idx]['modismo'],
            'modelo': model,
            'chrf_score': float(score),
            'definicion_real': referencias[idx],
            'definicion_generada': candidatos[idx]
        })

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_1_chrf_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_chrf, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS con chrF:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_chrf if r['modelo'] == model]
    if model_results:
        chrf_scores = [r['chrf_score'] for r in model_results]
        chrf_mean = np.mean(chrf_scores)
        chrf_std = np.std(chrf_scores)
        
        print(f"   {model}:")
        print(f"      • chrF: {chrf_mean:.4f} (±{chrf_std:.4f})")

print(f"\n✓ Guardado en: {output_file}")

print("\n" + "="*80)
print("PROMPT 1 COMPLETADO")
print("="*80 + "\n")

PROMPT 1: Modismo → Definición
Objetivo: Evaluar si el modelo puede generar una definición correcta del modismo
Métricas:
  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT
  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet
  - chrF: Character n-gram F-score
--------------------------------------------------------------------------------
Datos cargados: 20 registros válidos de 20 totales


Calculando BERTScore con BETO
Evaluando modelo: amazon/nova-micro-v1
Evaluando modelo: cohere/command-r-08-2024
Evaluando modelo: cohere/command-r-08-2024

RESULTADOS con BETO:
   amazon/nova-micro-v1:
      • Precision: 0.4491
      • Recall:    0.4305
      • F1 Score:  0.4389 (±0.0617)
   cohere/command-r-08-2024:
      • Precision: 0.4583
      • Recall:    0.4338
      • F1 Score:  0.4454 (±0.0906)

✓ Guardado en: Metricas LLM/Metricas_Resultados/prompt_1_beto_bertscore_resultados.json

Calculando BERTScore con SciBETO
Evaluando modelo: amazon/nova-micro

## 2. PROMPT 2: Modismo → Es Modismo (Sí/No)

**Métrica**: Accuracy  
**Justificación**: Es una tarea de clasificación binaria. Mide si el modelo identifica correctamente que una expresión es un modismo.

In [9]:
print("="*80)
print("PROMPT 2: Modismo → ¿Es Modismo? (Sí/No)")
print("="*80)
print("Objetivo: Evaluar si el modelo identifica correctamente que una expresión es un modismo")
print("Métrica: Accuracy (Exactitud)")
print("Justificación: Problema de clasificación binaria (Sí/No).")
print("   Mide el porcentaje de respuestas correctas.")
print("-"*80)

# Cargar datos desde JSON
with open(os.path.join(DATA_DIR, 'prompt_2_metrics_data.json'), 'r', encoding='utf-8') as f:
    data_p2 = json.load(f)

# Filtrar datos vacíos
data_p2_valid = [d for d in data_p2 if d.get('es_modismo_real') and d.get('es_modismo_generado')]

print(f"Datos cargados: {len(data_p2_valid)} registros válidos de {len(data_p2)} totales")
print()

def normalizar_respuesta(respuesta):
    """Normaliza respuestas a 'Sí' o 'No'"""
    if not respuesta:
        return None
    respuesta = str(respuesta).strip().lower()
    if respuesta in ['sí', 'si', 'yes', 's', 'true', '1']:
        return 'Sí'
    elif respuesta in ['no', 'n', 'false', '0']:
        return 'No'
    return respuesta

resultados_p2 = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")
    
    # Filtrar datos de este modelo
    model_data = [d for d in data_p2_valid if d.get('modelo') == model]
    
    if not model_data:
        print(f"  ⚠ No hay datos para {model}")
        continue
    
    # Calcular accuracy
    correctos = 0
    for d in model_data:
        real = normalizar_respuesta(d['es_modismo_real'])
        generado = normalizar_respuesta(d['es_modismo_generado'])
        correcto = (real == generado)
        
        if correcto:
            correctos += 1
        
        resultados_p2.append({
            'modismo': d['modismo'],
            'modelo': model,
            'respuesta_real': real,
            'respuesta_generada': generado,
            'correcto': correcto
        })
    
    accuracy = correctos / len(model_data) if model_data else 0
    print(f"  • Accuracy: {accuracy:.4f} ({correctos}/{len(model_data)} correctos)")

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_2_accuracy_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_p2, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_p2 if r['modelo'] == model]
    if model_results:
        correctos = sum(1 for r in model_results if r['correcto'])
        total = len(model_results)
        accuracy = correctos / total if total > 0 else 0
        print(f"   {model}:")
        print(f"      • Accuracy:  {accuracy:.4f} ({correctos}/{total} correctos)")

print()
print(f"✓ Guardado en: {output_file}")
print("="*80)
print()

PROMPT 2: Modismo → ¿Es Modismo? (Sí/No)
Objetivo: Evaluar si el modelo identifica correctamente que una expresión es un modismo
Métrica: Accuracy (Exactitud)
Justificación: Problema de clasificación binaria (Sí/No).
   Mide el porcentaje de respuestas correctas.
--------------------------------------------------------------------------------
Datos cargados: 20 registros válidos de 20 totales

Evaluando modelo: amazon/nova-micro-v1
  • Accuracy: 0.1000 (1/10 correctos)
Evaluando modelo: cohere/command-r-08-2024
  • Accuracy: 0.5000 (5/10 correctos)

RESULTADOS:
   amazon/nova-micro-v1:
      • Accuracy:  0.1000 (1/10 correctos)
   cohere/command-r-08-2024:
      • Accuracy:  0.5000 (5/10 correctos)

✓ Guardado en: Metricas LLM/Metricas_Resultados/prompt_2_accuracy_resultados.json



## 3. PROMPT 3: Modismo + Ejemplo → Literal + Definición

**Métrica**: BERTScore (F1)  
**Justificación**: Evalúa si el modelo puede generar interpretaciones literales del modismo en contexto.

In [10]:
print("="*80)
print("PROMPT 3: Modismo + Ejemplo → Literal + Definición")
print("="*80)
print("Objetivo: Evaluar si el modelo genera interpretaciones literales correctas")
print("Métricas:")
print("  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT")
print("  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet")
print("  - chrF: Character n-gram F-score")
print("-"*80)

# Cargar datos desde JSON
with open(os.path.join(DATA_DIR, 'prompt_3_metrics_data.json'), 'r', encoding='utf-8') as f:
    data_p3 = json.load(f)

# Filtrar datos con literal y definición generados
data_p3_valid = [d for d in data_p3 if d.get('significado_real') and d.get('definicion_generada')]

print(f"Datos cargados: {len(data_p3_valid)} registros válidos de {len(data_p3)} totales")
print()

# ============================================================================
# 1. BERTScore con BETO y SciBETO
# ============================================================================
bert_models = [
    ('BETO', compute_bertscore_beto),
    ('SciBETO', compute_bertscore_sci_beto)
]

for bert_name, bert_func in bert_models:
    print(f"\n{'='*60}")
    print(f"Calculando BERTScore con {bert_name}")
    print('='*60)
    
    resultados_p3 = []
    
    for model in MODEL_NAMES:
        print(f"Evaluando modelo: {model}")
        
        # Filtrar datos de este modelo
        model_data = [d for d in data_p3_valid if d.get('modelo') == model]
        
        if not model_data:
            print(f"  ⚠ No hay datos para {model}")
            continue
        
        referencias = [d['significado_real'] for d in model_data]
        candidatos = [d['definicion_generada'] for d in model_data]
        
        # BERTScore: compara definición generada vs significado real
        P, R, F1 = bert_func(candidatos, referencias)
        
        for idx, (p, r, f1) in enumerate(zip(P.tolist(), R.tolist(), F1.tolist())):
            resultados_p3.append({
                'modismo': model_data[idx]['modismo'],
                'ejemplo': model_data[idx]['ejemplo'],
                'modelo': model,
                'bert_model': bert_name,
                'precision': p,
                'recall': r,
                'f1_score': f1,
                'significado_real': referencias[idx],
                'literal_generado': model_data[idx].get('literal_generado', ''),
                'definicion_generada': candidatos[idx]
            })
    
    # Guardar resultados
    output_file = os.path.join(OUTPUT_DIR, f'prompt_3_{bert_name.lower()}_bertscore_resultados.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(resultados_p3, f, ensure_ascii=False, indent=2)
    
    print()
    print(f"RESULTADOS con {bert_name}:")
    for model in MODEL_NAMES:
        model_results = [r for r in resultados_p3 if r['modelo'] == model]
        if model_results:
            f1_scores = [r['f1_score'] for r in model_results]
            precisions = [r['precision'] for r in model_results]
            recalls = [r['recall'] for r in model_results]
            
            f1_mean = np.mean(f1_scores)
            f1_std = np.std(f1_scores)
            p_mean = np.mean(precisions)
            r_mean = np.mean(recalls)
            
            print(f"   {model}:")
            print(f"      • Precision: {p_mean:.4f}")
            print(f"      • Recall:    {r_mean:.4f}")
            print(f"      • F1 Score:  {f1_mean:.4f} (±{f1_std:.4f})")
    
    print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 2. Sentence-BERT Similarity
# ============================================================================
print(f"\n{'='*60}")
print(f"Calculando Sentence-BERT Similarity")
print('='*60)

resultados_sbert = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")
    
    model_data = [d for d in data_p3_valid if d.get('modelo') == model]
    
    if not model_data:
        print(f"  ⚠ No hay datos para {model}")
        continue
    
    referencias = [d['significado_real'] for d in model_data]
    candidatos = [d['definicion_generada'] for d in model_data]
    
    # Calcular similitud con Sentence-BERT
    similarities = compute_sbert_similarity(candidatos, referencias)
    
    for idx, sim in enumerate(similarities):
        resultados_sbert.append({
            'modismo': model_data[idx]['modismo'],
            'ejemplo': model_data[idx]['ejemplo'],
            'modelo': model,
            'similarity': float(sim),
            'significado_real': referencias[idx],
            'literal_generado': model_data[idx].get('literal_generado', ''),
            'definicion_generada': candidatos[idx]
        })

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_3_sbert_similarity_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_sbert, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS con Sentence-BERT:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_sbert if r['modelo'] == model]
    if model_results:
        similarities = [r['similarity'] for r in model_results]
        sim_mean = np.mean(similarities)
        sim_std = np.std(similarities)
        
        print(f"   {model}:")
        print(f"      • Similitud: {sim_mean:.4f} (±{sim_std:.4f})")

print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 3. chrF Score
# ============================================================================
print(f"\n{'='*60}")
print(f"Calculando chrF Score")
print('='*60)

resultados_chrf = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")
    
    model_data = [d for d in data_p3_valid if d.get('modelo') == model]
    
    if not model_data:
        print(f"  ⚠ No hay datos para {model}")
        continue
    
    referencias = [d['significado_real'] for d in model_data]
    candidatos = [d['definicion_generada'] for d in model_data]
    
    # Calcular chrF
    chrf_scores = compute_chrf_batch(candidatos, referencias)
    
    for idx, score in enumerate(chrf_scores):
        resultados_chrf.append({
            'modismo': model_data[idx]['modismo'],
            'ejemplo': model_data[idx]['ejemplo'],
            'modelo': model,
            'chrf_score': float(score),
            'significado_real': referencias[idx],
            'literal_generado': model_data[idx].get('literal_generado', ''),
            'definicion_generada': candidatos[idx]
        })

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_3_chrf_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_chrf, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS con chrF:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_chrf if r['modelo'] == model]
    if model_results:
        chrf_scores = [r['chrf_score'] for r in model_results]
        chrf_mean = np.mean(chrf_scores)
        chrf_std = np.std(chrf_scores)
        
        print(f"   {model}:")
        print(f"      • chrF: {chrf_mean:.4f} (±{chrf_std:.4f})")

print(f"\n✓ Guardado en: {output_file}")

print("\n" + "="*80)
print("PROMPT 3 COMPLETADO")
print("="*80 + "\n")

PROMPT 3: Modismo + Ejemplo → Literal + Definición
Objetivo: Evaluar si el modelo genera interpretaciones literales correctas
Métricas:
  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT
  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet
  - chrF: Character n-gram F-score
--------------------------------------------------------------------------------
Datos cargados: 18 registros válidos de 20 totales


Calculando BERTScore con BETO
Evaluando modelo: amazon/nova-micro-v1
Evaluando modelo: cohere/command-r-08-2024
Evaluando modelo: cohere/command-r-08-2024

RESULTADOS con BETO:
   amazon/nova-micro-v1:
      • Precision: 0.5324
      • Recall:    0.5194
      • F1 Score:  0.5256 (±0.0741)
   cohere/command-r-08-2024:
      • Precision: 0.5483
      • Recall:    0.5307
      • F1 Score:  0.5387 (±0.1035)

✓ Guardado en: Metricas LLM/Metricas_Resultados/prompt_3_beto_bertscore_resultados.json

Calculando BERTScore con SciBETO
Evaluando modelo: ama